<a href="https://colab.research.google.com/github/vm1729/Openthink/blob/master/TextNLP/101_Patent_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Similarity index



In [0]:
from __future__ import division
import string
import math

tokenize = lambda doc: doc.lower().split(" ")

In [0]:
d1="Method of fabricating an assembly comprising spraying a deposit forming material (1249) to at least a portion of the periphery of a glass pane unit, e.g. an insulating unit (1208). Masks (1237, 1237') are first applied to the outside surfaces of the panes (1220, 1222). The deposit (1249) may also cover the edge seal of the insulating glazing unit."
d2="PROBLEM TO BE SOLVED: To provide a method and apparatus for producing a heat insulating glass unit.SOLUTION: According to the apparatus and the method using the apparatus, an adhesive is applied to an edge portion of the heat insulating glass unit by means of one or more nozzles, and a device having the nozzles for applying the adhesive bead to a side surface of a window sheet to be mounted in a frame or a sash, and a side surface of the heat insulating glass unit have separate side arms extending therefrom. The method for producing the heat insulating glass unit is available for applying an application agent-forming material to the edge of the window sheet of the heat insulating glass unit."
d3="The present invention provides methods and apparatus for manufacturing insulating glass units. In apparatus and method of using the apparatus is provided by which adhesive is applied to the end regions of the insulating glass unit with one or more nozzles while a separate side arm extends from the device and alongside the insulating glass unit having a nozzle that delivers adhesive bead to the side of the pane to be adhered to the frame or sash. Methods are also provided for applying a deposit forming material to the periphery of the panes of an insulating glass unit."
d4="A glass article which has a water-sheeting coating and a method of applying coatings to opposed sides of a substrate are described. In one embodiment, a water-sheeting coating (20) comprising silica is sputtered directly onto an exterior surface of the glass. The exterior face of this water-sheeting coating is substantially non-porous but has an irregular surface. This water-sheeting coating causes water applied to the coated surface to sheet, making the glass article easier to clean and helping the glass stay clean longer. In one method of the invention, interior and exterior surfaces of a glass sheet are cleaned. Thereafter, the interior surface of the sheet of glass is coated with a reflective coating by sputtering, in sequence, at least one dielectric layer, at least one metal layer, and at least one dielectric layer. The exterior surface of the glass is coated with a water-sheeting coating by sputtering silica directly onto the exterior surface of the sheet of glass. If so desired, both the interior surface and the exterior surface can be applied during the same pass through the same sputter coating apparatus while the glass maintains a constant orientation wherein the interior surface is positioned above the exterior surface."
d5="Vladimir Putin was found to be riding a horse, again, without a shirt on while hunting deer. Vladimir Putin always seems so serious about things - even riding horses."
all_documents = [d1,d2,d3,d4,d5]

tokenized_documents = [tokenize(d) for d in all_documents] # tokenized docs
all_tokens_set = set([item for sublist in tokenized_documents for item in sublist])


In [0]:
document_0 = "China has a strong economy that is growing at a rapid pace. However politically it differs greatly from the US Economy."
document_1 = "At last, China seems serious about confronting an endemic problem: domestic violence and corruption."
document_2 = "Japan's prime minister, Shinzo Abe, is working towards healing the economic turmoil in his own country for his view on the future of his people."
document_3 = "Vladimir Putin is working hard to fix the economy in Russia as the Ruble has tumbled."
document_4 = "What's the future of Abenomics? We asked Shinzo Abe for his views"
document_5 = "Obama has eased sanctions on Cuba while accelerating those against the Russian Economy, even as the Ruble's value falls almost daily."
document_6 = "Vladimir Putin was found to be riding a horse, again, without a shirt on while hunting deer. Vladimir Putin always seems so serious about things - even riding horses."

all_documents = [document_0, document_1, document_2, document_3, document_4, document_5, document_6]

tokenized_documents = [tokenize(d) for d in all_documents] # tokenized docs
all_tokens_set = set([item for sublist in tokenized_documents for item in sublist])

In [0]:
def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

In [15]:
# comparing document_2 and document_4
jaccard_similarity(tokenized_documents[1],tokenized_documents[4])

0.038461538461538464

These documents have nothing in common. However they are rated as 8% relevant. This examples identifies the fundamental issues with Jaccard Similarity:

Length is irrelevant. (bias towards longer documents).
Words that appear in a lot of documents are weighted the same as those that appear in few. (bias towards longer documents as well as non-descriptive words)

#TF-IDF

In [0]:
def term_frequency(term, tokenized_document):
    return tokenized_document.count(term)

In [0]:
def sublinear_term_frequency(term, tokenized_document):
    return 1 + math.log(tokenized_document.count(term))

def augmented_term_frequency(term, tokenized_document):
    max_count = max([term_frequency(t, tokenized_document) for t in tokenized_document])
    return (0.5 + ((0.5 * term_frequency(term, tokenized_document))/max_count))

In [0]:
def inverse_document_frequencies(tokenized_documents):
    idf_values = {}
    all_tokens_set = set([item for sublist in tokenized_documents for item in sublist])
    for tkn in all_tokens_set:
        contains_token = map(lambda doc: tkn in doc, tokenized_documents)
        idf_values[tkn] = 1 + math.log(len(tokenized_documents)/(sum(contains_token)))
    return idf_values

In [18]:
idf_values = inverse_document_frequencies(tokenized_documents)
print (idf_values['insulating'])
# 2.94591014906
#print (idf_values['the'])
# 1.33647223662

1.5108256237659907


In [0]:
def tfidf(documents):
    tokenized_documents = [tokenize(d) for d in documents]
    idf = inverse_document_frequencies(tokenized_documents)
    tfidf_documents = []
    for document in tokenized_documents:
        doc_tfidf = []
        for term in idf.keys():
            tf = sublinear_term_frequency(term, document)
            doc_tfidf.append(tf * idf[term])
        tfidf_documents.append(doc_tfidf)
    return tfidf_documents

In [24]:
all_documents

["Method of fabricating an assembly comprising spraying a deposit forming material (1249) to at least a portion of the periphery of a glass pane unit, e.g. an insulating unit (1208). Masks (1237, 1237') are first applied to the outside surfaces of the panes (1220, 1222). The deposit (1249) may also cover the edge seal of the insulating glazing unit.",
 'PROBLEM TO BE SOLVED: To provide a method and apparatus for producing a heat insulating glass unit.SOLUTION: According to the apparatus and the method using the apparatus, an adhesive is applied to an edge portion of the heat insulating glass unit by means of one or more nozzles, and a device having the nozzles for applying the adhesive bead to a side surface of a window sheet to be mounted in a frame or a sash, and a side surface of the heat insulating glass unit have separate side arms extending therefrom. The method for producing the heat insulating glass unit is available for applying an application agent-forming material to the edg

In [23]:
tfidf_representation = tfidf(all_documents)
#print (tfidf_representation[0]), document_0

ValueError: ignored

# Sklearn TF-IDF

In [29]:
all_documents

["Method of fabricating an assembly comprising spraying a deposit forming material (1249) to at least a portion of the periphery of a glass pane unit, e.g. an insulating unit (1208). Masks (1237, 1237') are first applied to the outside surfaces of the panes (1220, 1222). The deposit (1249) may also cover the edge seal of the insulating glazing unit.",
 'PROBLEM TO BE SOLVED: To provide a method and apparatus for producing a heat insulating glass unit.SOLUTION: According to the apparatus and the method using the apparatus, an adhesive is applied to an edge portion of the heat insulating glass unit by means of one or more nozzles, and a device having the nozzles for applying the adhesive bead to a side surface of a window sheet to be mounted in a frame or a sash, and a side surface of the heat insulating glass unit have separate side arms extending therefrom. The method for producing the heat insulating glass unit is available for applying an application agent-forming material to the edg

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(min_df=1, stop_words="english") 
tfidf = vect.fit_transform(all_documents)
pairwise_similarity = tfidf * tfidf.T 

In [34]:
pairwise_similarity.toarray()

array([[1.        , 0.28453253, 0.3219071 , 0.04962132, 0.        ],
       [0.28453253, 1.        , 0.50359902, 0.19400144, 0.        ],
       [0.3219071 , 0.50359902, 1.        , 0.11021245, 0.        ],
       [0.04962132, 0.19400144, 0.11021245, 1.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 1.        ]])